# Uniview module for IceCube event 170922A

*Aaron Geller, 2018*

### Imports and function definitions

In [109]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template 

In [206]:
Template = """mesh
{  
    #data  ICgeom ./modules/$folderName/IceCubeGeometry.uvstream
    data ICgeom    ./modules/$folderName/IceCubeGeometry.raw
    data ICevent    ./modules/$folderName/ic170922.tsort.raw # cat ic170922.raw | sort -n -k 4 > ic170922.tsort.raw


    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec1f geomRadius 10 | public | desc "geometry point size " | widget slider | range 0 20
            vec1f eventRadius 15 | public | desc "event scale point size " | widget slider | range 0 20
            vec1f eventTime 0 | public | desc "event time " | widget slider | range 0 11800
            vec1f tfac 5 | public | desc "time width " | widget slider | range 0. 20
            
        }
    }
    ############# Detector geometry
    pass
    {
        useDataObject ICgeom
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/ICgeom.gs
                vertexShader ./modules/$folderName/ICgeom.vs
                fragmentShader   ./modules/$folderName/ICgeom.fs
                
                stateManagerVar __objectName__.geomRadius  geomRadius
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    ############# Event geometry
    pass
    {
        useDataObject ICevent
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/ICevent.gs
                vertexShader ./modules/$folderName/ICevent.vs
                fragmentShader   ./modules/$folderName/ICevent.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    #wrapModeS GL_CLAMP_TO_EDGE
                    #wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }
                
                stateManagerVar __objectName__.eventRadius  eventRadius
                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.eventTime  tfac
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
}"""

### UVStream for IceCubeGeometry

In [207]:
ICG_UVSTREAM='''<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<DataStream version="0.1">
    <PrimitiveType>line_strip</PrimitiveType>
    <DataBindings>
        <Variable>
            <Name>vertex</Name>         
            <Type>vec3f</Type>
        </Variable>                        
    </DataBindings>

    <CSVData>
        <Path>IceCubeGeometry.csv</Path>
        <Headers>
            <Variable>              
                <Cols>0,1,2</Cols>
            </Variable>                                       
         </Headers>
    </CSVData>
</DataStream>  
'''

In [208]:
uvStreamFile=open('data/IceCubeGeometry.uvstream','w')
uvStreamFile.write(ICG_UVSTREAM)
uvStreamFile.close()

### IceCube class

In [209]:
class IceCube():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("ICgeom.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("ICgeom.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("ICgeom.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("ICevent.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("ICevent.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("ICevent.fs",dataFolder)
        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [210]:
model = IceCube(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [211]:
model.object.setcameraradius(10)
model.object.setcoord("Earth")
model.object.setname("IceCube")
model.object.setguiname("/KavliLecture/Larson/IceCube")
model.object.settargetradius(20)
model.object.showatstartup(False)
model.cr = 10000
modinfo.setname("IceCube")
modinfo.setauthor("Aaron Geller<sup>1</sup> and Shane Larson<sup>2</sup><br />(1)Adler Planetarium,br />(2)Northwestern University")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for IceCube event 170922A")
#modinfo.setthumbnail("data/R0010133.JPG")
modinfo.setversion("1.0")

generator.generate("IceCube",[],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

## Helper Functions for modifing code
*Reload Module and Shaders in Uniview*

In [179]:
uvmod.Utility.senduvcommand(model.object.name+".reload; system.reloadallshaders")

*Copy modified Shader files and reload*

In [52]:
from config import Settings


In [67]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")

### Create colormap texture

In [103]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [141]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig("data/cmap.png", bbox_inches='tight',pad_inches=0)

plot_cmap('viridis')